# AgentOps Observability and Monitoring
Using [AgentOps](https://agentops.ai/) allows you to quickly and easily add agent observability and monitoring to your Autogen services.

````{=mdx}
:::info Requirements
Some extra dependencies are needed for this notebook, which can be installed via pip:

```bash
pip install pyautogen[agentops]
```

For more information, please refer to the [installation guide](/docs/installation/).
:::
````

### Adding AgentOps to an existing Autogen service.
It's as simple as adding two lines of code!

In [1]:
%pip install agentops

Note: you may need to restart the kernel to use updated packages.


In [2]:
from typing import Annotated, Literal
Operator = Literal["+", "-", "*", "/"]

In [3]:
def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")

In [4]:
import agentops
from autogen import ConversableAgent, UserProxyAgent, config_list_from_json, register_function

# agentops.init(api_key="7c94212b-b89d-47a6-a20c-23b2077d3226", tags=["autogen-callback-example"])
agentops.init(api_key="6b7a1469-bdcb-4d47-85ba-c4824bc8486e", endpoint='http://localhost:8000', tags=["autogen-callback-example"])

config_list = config_list_from_json(env_or_file="OAI_CONFIG_LIST")

assistant = ConversableAgent(
        name="Assistant",
        system_message="You are a helpful AI assistant. "
        "You can help with simple calculations. "
        "Return 'TERMINATE' when the task is done.",
        llm_config={"config_list": config_list, "cache_seed": None},
    )

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

assistant.register_for_llm(name="calculator", description="A simple calculator")(calculator)
user_proxy.register_for_execution(name="calculator")(calculator)

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",  # A description of the tool.
)

# Let the assistant start the conversation.  It will end when the user types exit.
user_proxy.initiate_chat(assistant, message="What is (1423 - 123) / 3 + (32 + 23) * 5?")

agentops.end_session("Success")


Starting agentops logger
User (to Assistant):

What is (1423 - 123) / 3 + (32 + 23) * 5?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_HF67N53zJqjXLIxMybAn02tk): calculator *****
Arguments: 
{
  "a": 1423,
  "b": 123,
  "operator": "-"
}
***************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION calculator...
User (to Assistant):

User (to Assistant):

***** Response from calling tool (call_HF67N53zJqjXLIxMybAn02tk) *****
1300
**********************************************************************

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Assistant (to User):

***** Suggested tool call (call_L8ZEWmUPW394YeRXbczBM5zw): calculator *****
Arguments: 
{
  "a": 1300,
  "b": 3,
  "